In [ ]:
from bs4 import BeautifulSoup as bs
import lxml
import nltk
import os
import string

nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
def extract_info(filepath,words):
    
    dictionary_of_interest = {}
    
    with open(filepath,"r",encoding="utf8") as file:
            
        content = file.readlines()
        content = "".join(content)
        
        bs_content = bs(content, "lxml")
        
        unique_id = bs_content.find("tei").attrs["xml:id"]
        
        dictionary_of_interest["unique_id"] = unique_id
        
        letter_details = bs_content.find_all("correspaction")
        
        for deets in letter_details:
            
            if deets.attrs["type"] == "sent":
                
                if "when" in list(deets.date.attrs.keys()):
                    dictionary_of_interest["date"] = deets.date.attrs["when"]
                
                try:
                    dictionary_of_interest["sender"] = deets.persname.text
                except AttributeError:
                    dictionary_of_interest["reciever"] = deets.orgname.text
                
                try:
                    dictionary_of_interest["sender_bio"] = deets.persname.attrs["key"]
                except AttributeError:
                    dictionary_of_interest["sender_bio"] = "None Available"
                except KeyError:
                    dictionary_of_interest["sender_bio"] = "None Available"
                    
            if deets.attrs["type"] == "received":
                
                try:
                    dictionary_of_interest["reciever"] = deets.persname.text
                except AttributeError:
                    dictionary_of_interest["reciever"] = deets.orgname.text
                    
                try:
                    dictionary_of_interest["reciever_bio"] = deets.persname.attrs["key"]
                except AttributeError:
                    dictionary_of_interest["reciever_bio"] = "None Available"
                except KeyError:
                    dictionary_of_interest["reciever_bio"] = "None Available"
                    
        try:
            free_text = bs_content.find_all("div",{"type":"transcription"})[0].p.text
        except AttributeError:
#             print(bs_content) 
            free_text = ""
            

        # cleaning of the data
        dictionary_of_interest["body"] = (free_text.lower()).translate(str.maketrans('','',string.punctuation))
        text_tokens = word_tokenize(dictionary_of_interest["body"])
#         no_stop_words = [word for word in text_tokens] # if not word in stopwords.words()
        for word in text_tokens:
            words.append(word)
#         print(no_stop_words)

        file.close()
        
    return dictionary_of_interest

def generate_feature_data(free_text,feature_set):
    
    feature_bools = []
    
    for word in feature_set:
        feature_bools.append(1*(word in free_text))
        
    return feature_bools

In [ ]:
path = "dcp-data/letters/"
files = os.listdir(path)
files = files[1:]
words = []
# print(files[1])
i = 0
cap = 10000

if cap == 0:
    cap = len(files)
    
for file_target in files:
    dict_test = extract_info(path+file_target,words)
#     print(dict_test)
    if i == cap:
        break
    elif i < cap:
        i += 1
    else:
        print("Failed loop")
        break
    print((i/cap)*100,end="\r"*(i!=cap))

print("Analysis Finished")
print(len(words))

In [ ]:
unique_words = {}
for counter, word in enumerate(words):
    if word not in list(unique_words.keys()):
        unique_words[word] = 1
    else:
        unique_words[word] += 1
    print(round(((counter+1)/len(words))*100,2),end="\r")
    
sorted_unique_words = {key: value for key, value in sorted(unique_words.items(), key=lambda item: item[1],reverse=True)}

# print(list(sorted_unique_words.keys())[:1000])

In [ ]:
feature_words = list(sorted_unique_words.keys())[:4000]

In [ ]:
test = generate_feature_data(dict_test["body"],feature_words)
print(dict_test["sender"])
print(dict_test["reciever"])